# Ejercicios

* Crea una clase abstracta llamada CuentaBancaria que tenga un atributo saldo (Decimal). Luego, crea dos subclases de CuentaBancaria llamadas CuentaCorriente y CajaDeAhorro.
 + La clase CuentaCorriente debe tener un atributo adicional llamado limite_descubierto (Decimal).
 + Ambas subclases deben tener métodos para depositar y retirar dinero de la cuenta, así como para obtener el saldo actual.
 + El depósito funciona de la misma manera en ambos casos, pero el retiro en caso de Caja de ahorro no debe permitir llevar el saldo a negativo, mientras que el caso de Cuenta Corriente no debe permitir tener saldo negativo mayor al limite_descubierto.
 + El límite de descubierto puede ser modificado.
 + Se debe sobrecargar los operadores + y - que tomaran una cuenta y un monto y funcionarán como deposito y retiro respectivamente
 + La clase CuentaBancaria de tener un metodo de clase que devuelva al suma de todos los saldos de todas las cuentas creadas, según con que clase sea invocada.



 + Revisar y tratar de entender estas dos formas de definir una clase a la que al construir le puedo pasar parametros con nombre para obtener un objeto con esos atributos accesibles a través del uso del . como si fueran atributos definidos en la instancia.


In [ ]:
class Bunch(object):
  def __init__(self, **kwds):
    self.__dict__.update(kwds)

  def __eq__(self, other):
    return self.__dict__ == other.__dict__


class Bunch2(dict):
  __getattr__, __setatr__ = dict.get, dict.__setitem__

x = Bunch(pepe=23)
print(x.pepe)

print(x)

x = Bunch2(pepe=23,update=33)
print(x.pepe)
print(x)
print(x.update)

